# Project: Ear Recognition
Yifei Feng | U81901533

Zehui Jiang | U68975915

Guangxing Ren | U072735315

# Identify your problem
**Take a look at your topic and available resources, come up with a problem you think is interesting and worth working on. Give details about why is it a valuable topic and then describe the problem you are going to solve in a precise way.**

#AI recognization system has been well implementing these days. A famous commercial example is facing ID by Apple. Before unlocking the device or finish the transaction, a facial scan is being conducted to check the identification of the user.  Another similar area#

The process of precisely recognize people by ears also has been getting major attention in recent years. It represents an important step in the biometric research, especially as a complement to face recognition systems which have difficulty in real conditions. This is due to the great variation in shapes, variable lighting conditions, and the changing profile shape which is a planar representation of a complex object. We present an ear recognition system involving a convolutional neural network (CNN)  to identify a person given an input image.

There are few methods 
Data:
We are given a set of left ear image from people with different identities. For each ear(each person). Four images are given, we for ears “in the wild” (there is no constraint in the way of taking the photo), two for ear images taken with a “donut device” that serves as a background and somewhat controls lighting.  There are 4*195(individual ear) in the dataset.

 http://cs-people.bu.edu/wdqin/earImageDataset.zip
 
 
 
 


# Backgroud


Find out about and list more than one existing (possible) solution to your problem, make a brief comparison, and evaluation on these methods. Choose one of the existing solutions as a baseline for comparison.  Consider how much time you have and effort might be required to reproduce the result. Be realistic with your goal.


We found an existing Face Recognition CNN that applies Convolutional Neural Network. 

We use kears as our open-source neural-network library.Keras is a high-level API for neural networks. It is written in Python and its biggest advantage is its ability to run on top of state-of-art deep learning libraries/frameworks such as TensorFlow, CNTK or Theano.



These are the time distribution we planned for each individual member:

Background research:   4 hours
Reproducing Baseline: 6 hours
Improvement: 5  hours
Final Report: 4 hours

We don’t want to ignore the importance of background research and report. Since we are given limited time for this complicated project. We want to spend more time on understanding the existing algorithm and application.  


#  Reproducing the baseline:

We found an existing Face Recognition CNN  that applies Convolutional Neural Network. 


In [ ]:
from skimage.io import imread_collection
import cv2
import numpy as np
from keras.utils import np_utils 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

#loading dataset from directory and resize every picture to 200x200 dmension
def load_data(path, size):
    #creating a collection with the available images
    image = imread_collection(path)
    image_set = []
    for n in image:
        n = cv2.cvtColor(n,cv2.COLOR_RGB2GRAY)
        n = cv2.resize(n,(size,size)) 
        n = n / 255
        image_set.append(n)
    return image_set 
    
def set_init(dataset, train_set_ratio, valid_set_ratio, test_set_ratio):

    
    #creating label set for all images
    label = np.empty(195*4)
    for i in range(195):
        label[i*4:i*4+4] = i
    label = label.astype(np.int)
    label = np_utils.to_categorical(label, 195)#transfer to one-hot matrix
    
    train_num = 780*train_set_ratio
    train_num = int(train_num)
    valid_num = 780*valid_set_ratio
    valid_num = int(valid_num)
    test_num = 780*test_set_ratio
    test_num = int(test_num) 
    
    train_data = np.empty((train_num,200,200))  #creating numpy array for different datasets
    train_label = np.empty((train_num,195))   
    valid_data = np.empty((valid_num, 200,200))   
    valid_label = np.empty((valid_num,195))   
    test_data = np.empty((test_num,200,200))  
    test_label = np.empty((test_num,195)) 
    
    x_test_tot = np.empty((valid_num + test_num,200,200))
    y_test_tot = np.empty((valid_num + test_num,195))
    
    train_data, x_test_tot, train_label, y_test_tot = train_test_split(dataset, label, test_size = 1-train_set_ratio)
    
    valid_data, test_data, valid_label, test_label = train_test_split(x_test_tot, y_test_tot, test_size = test_set_ratio/(valid_set_ratio + test_set_ratio))
    
    result = [(train_data, train_label), (valid_data, valid_label),(test_data, test_label)]
   
    return result

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D,AveragePooling2D
from PIL import Image
def train(data, batch_size, epochs, nb_filters, pool_size, kernel_size):
    np.random.seed(1337)  # for reproducibility
    img_rows, img_cols = 200, 200  # 输入图片样本的宽高
    nb_classes = 195  # 分类数目
    input_shape = (img_rows, img_cols,1)  # 输入图片的维度

    [(X_train, Y_train), (X_valid, Y_valid),(X_test, Y_test)] = data

    X_train = X_train[:,:,:,np.newaxis]  # 添加一个维度，代表图片通道。这样数据集共4个维度，样本个数、宽度、高度、通道数
    X_valid=X_valid[:,:,:,np.newaxis]  # 添加一个维度，代表图片通道。这样数据集共4个维度，样本个数、宽度、高度、通道数
    X_test=X_test[:,:,:,np.newaxis]  # 添加一个维度，代表图片通道。这样数据集共4个维度，样本个数、宽度、高度、通道数
    print('样本数据集的维度：', X_train.shape,Y_train.shape)
    print('测试数据集的维度：', X_test.shape,Y_test.shape)
    model = Sequential()
    model.add(Conv2D(6,kernel_size,input_shape=input_shape,strides=1))  # 卷积层1
    model.add(AveragePooling2D(pool_size,strides=2))  # 池化层
    model.add(Conv2D(12,kernel_size,strides=1))  # 卷积层2
    model.add(AveragePooling2D(pool_size,strides=2))  # 池化层
    model.add(Flatten())  # 拉成一维数据
    model.add(Dense(nb_classes))  # 全连接层2
    model.add(Activation('sigmoid'))  # sigmoid评分

    # 编译模型
    model.compile(loss='categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])
    # 训练模型
    model.fit(X_train, Y_train, batch_size, epochs,verbose=1, validation_data=(X_valid, Y_valid))
    # 评估模型
    score = model.evaluate(X_test, Y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])



    y_pred = model.predict(X_test)
    y_pred = y_pred.argmax(axis=1)   # 获取概率最大的分类，获取每行最大值所在的列
    for i in range(len(y_pred)):
    #     oneimg = X_test[i,:,:,0]*256
    #     im = Image.fromarray(oneimg)
    #     im.show()
        print('第%d个人识别为第%d个人'%(i,y_pred[i]))

# Background survey

# Reproducing the baseline

# Improvement
## 1. Regularization
## 2. Augmentation
## 3. Crop
## 4. k-fold
## 5. Dropout
## 6. Comprise